In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 10
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()
sys.box = box
sys.atoms = satoms


2000


In [3]:
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6)

In [31]:
qlist = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 
         20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
        31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
        41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

In [32]:
%%timeit
sys.calculate_q(qlist, use_c=False)

18.3 s ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
sys.calculate_q(qlist, use_c=True)

13.1 s ± 231 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
sys.atoms.keys()

dict_keys(['positions', 'ids', 'types', 'ghost', 'neighbors', 'neighbor_dist', 'temp_neighbors', 'temp_neighbordist', 'neighborweight', 'diff', 'r', 'theta', 'phi', 'cutoff', 'neighbordist', 'q2', 'q2_real', 'q2_imag', 'q0', 'q0_real', 'q0_imag', 'q1', 'q1_real', 'q1_imag', 'q3', 'q3_real', 'q3_imag', 'q4', 'q4_real', 'q4_imag', 'q5', 'q5_real', 'q5_imag', 'q6', 'q6_real', 'q6_imag', 'q7', 'q7_real', 'q7_imag', 'q8', 'q8_real', 'q8_imag', 'q9', 'q9_real', 'q9_imag', 'q10', 'q10_real', 'q10_imag', 'q11', 'q11_real', 'q11_imag', 'q12', 'q12_real', 'q12_imag', 'q13', 'q13_real', 'q13_imag', 'q14', 'q14_real', 'q14_imag', 'q15', 'q15_real', 'q15_imag', 'q16', 'q16_real', 'q16_imag', 'q17', 'q17_real', 'q17_imag', 'q18', 'q18_real', 'q18_imag', 'q19', 'q19_real', 'q19_imag', 'q20', 'q20_real', 'q20_imag', 'q21', 'q21_real', 'q21_imag', 'q22', 'q22_real', 'q22_imag', 'q23', 'q23_real', 'q23_imag', 'q24', 'q24_real', 'q24_imag', 'q25', 'q25_real', 'q25_imag', 'q26', 'q26_real', 'q26_imag', 'q